In [1]:
import os
from rdkit.Chem import AllChem
from rdkit import Chem
import torch
import numpy as np
from torch_geometric.data import Data

# from torch.utils.data import DataLoader

# from torch_geometric.loader.dataloader import DataLoader
from torch_geometric.data import DataLoader
# from dataloader import DataLoaderSubstructContext

from model import GNN, GNN_graphpred

from tqdm import tqdm

# from torchsummary import summary
# from loader import MoleculeDataset
from loader import CustomDataset

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# torch.autograd.set_detect_anomaly(True)

In [3]:
device = torch.device("cuda:0")
print(device)

cuda:0


In [4]:
# dataset = MoleculeDataset("data/", dataset="mutag")
dataset = CustomDataset(root="data")
loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
model = GNN_graphpred(num_layer=5, emb_dim=300, num_tasks=2, JK="last", drop_ratio=0.5, graph_pooling="mean")
model.from_pretrained("../chem/model_gin/supervised.pth")
model.to(device)

GNN_graphpred(
  (gnn): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0): GINConv(
        (mlp): Sequential(
          (0): Linear(in_features=300, out_features=600, bias=True)
          (1): ReLU()
          (2): Linear(in_features=600, out_features=300, bias=True)
        )
        (edge_embedding1): Embedding(6, 300)
        (edge_embedding2): Embedding(3, 300)
      )
      (1): GINConv(
        (mlp): Sequential(
          (0): Linear(in_features=300, out_features=600, bias=True)
          (1): ReLU()
          (2): Linear(in_features=600, out_features=300, bias=True)
        )
        (edge_embedding1): Embedding(6, 300)
        (edge_embedding2): Embedding(3, 300)
      )
      (2): GINConv(
        (mlp): Sequential(
          (0): Linear(in_features=300, out_features=600, bias=True)
          (1): ReLU()
          (2): Linear(in_features=600, out_features=300, bias=True)
        )
        (edge_embedding1): E

In [6]:
for step, batch in enumerate(tqdm(loader, desc="Iteration")):
    batch = batch.to(device)
    print(batch)
    with torch.no_grad():
        pred = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        print(pred.shape)

#     y_true.append(batch.y.view(pred.shape))
#     y_scores.append(pred)

Iteration: 100%|██████████| 2/2 [00:00<00:00, 47.20it/s]

Batch(batch=[7], edge_attr=[20, 2], edge_index=[2, 20], id=[1], x=[7, 2])
torch.Size([1, 300])
Batch(batch=[7], edge_attr=[16, 2], edge_index=[2, 16], id=[1], x=[7, 2])
torch.Size([1, 300])


In [7]:
# x_embedding1 = torch.nn.Embedding(120, 300)
# x_embedding2 = torch.nn.Embedding(40, 300)

# inp = torch.randint(3, 55, (6,))

# print("1", inp.shape)

# y1 = x_embedding1(inp)
# print("y1", y1.shape)

# print("2", inp.shape)

# y2 = x_embedding2(inp)
# print("3", inp.shape)
# print(y2.shape)